In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
gdrive_path = '/content/drive/MyDrive/thesis/'

In [1]:
%%capture
!pip install bertopic

In [21]:
from bertopic import BERTopic
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
import gensim
import spacy
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
import gensim.models as models
from textblob import TextBlob
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from ast import literal_eval
import string

In [3]:
df = pd.read_csv('Report Dataset.csv', encoding='latin-1')
df.sample(10)

,techreport,conclusion,Abnormal
9367,This EEG was done on a 25 years old cooperativ...,"Intermittent sharp waves seen bitemporally, wi...",NaN
12107,This EEG was done on a years old oxygen depend...,Abnormal EEG :\n1. Periodic complexes noted ov...,NaN
7779,This EEG was done on a cooperative patient but...,Abnormal EEG in view of absent of background a...,NaN
650,"During sleep recording, symmetrical vertex sha...",The EEG is within normal limits.\n(31/01/2004),0.0
651,The EEG was done on a persistently moved child...,This is an abnormal EEG with suggestion of foc...,1.0
3536,This EEG was done on a restless and confused p...,The EEG showed an absence of normal background...,1.0
9433,The EEG background was low in voltage. In wake...,"Some episode of theta wave at 5-7 Hz, amplitud...",NaN
5261,This EEG was done to 16 years old boy who was ...,EEG is normal.\n(21/04/2008),NaN
10299,"During spontaneous sleep, showed symmetrical v...",Agree with above.\n(16/03/2012),NaN
13794,This EEG was done on 39 year old cooperative p...,Normal awake and sleep EEG.,NaN


In [4]:
df = df.drop(columns=['techreport', 'Abnormal'])
df.dropna(inplace=True)
df.sample(10)

,conclusion
4115,1) Short run of rhythmic activities over (R) c...
7204,Abnormal EEG because of (R) centrotemporal sha...
7708,Abnormal EEG.\nThere was generalised attenuate...
4105,Abnormal EEG because of irregular activity of ...
5200,"Asymmetrical slow background, slower in the (R..."
4086,This is normal EEG.
11622,Fairly frequent sharp/spike with after coming ...
742,This EEG showed no evidence of definite epilep...
2794,Normal EEG.
3476,Abnormal EEG because of (R) anterior/ mid temp...


In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yijin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

In [7]:
# TEXT CLEANING
TEXT_CLEANING = "@\S+|https?:\S+|http?:\S|[^A-Za-z]+"

def preprocess(text, stem=False):

    text = re.sub(TEXT_CLEANING, ' ', str(text).lower()).strip()
    tokens = []

    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)

    return " ".join(tokens)

In [8]:
df.conclusion = df.conclusion.apply(lambda x: preprocess(x))

In [9]:
df.sample(10)

,conclusion
13615,abnormal eeg background theta activity hz comp...
257,eeg improved although still show encephalopath...
12705,abnormal awake sleep eeg showing spike polyspi...
7275,normal eeg
5388,l temporal sharp waves consistent partial seiz...
10957,diffuse low amplitude background activity cons...
12449,awake sleep eeg findings within normal limit
8293,eeg within normal limits
12395,eeg normal episodes movement felt seen recorde...
11303,diffuse polymorphic delta slow waves seen exce...


In [63]:
bertmodel = BERTopic(nr_topics = 15, verbose=True)
topics, probabilities = bertmodel.fit_transform(docs)

2024-06-04 10:37:26,033 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 466/466 [00:05<00:00, 87.01it/s] 
2024-06-04 10:37:33,632 - BERTopic - Embedding - Completed ✓
2024-06-04 10:37:33,634 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-04 10:37:43,769 - BERTopic - Dimensionality - Completed ✓
2024-06-04 10:37:43,770 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-06-04 10:37:44,149 - BERTopic - Cluster - Completed ✓
2024-06-04 10:37:44,150 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-06-04 10:37:44,355 - BERTopic - Representation - Completed ✓
2024-06-04 10:37:44,356 - BERTopic - Topic reduction - Reducing number of topics
2024-06-04 10:37:44,496 - BERTopic - Topic reduction - Reduced number of topics from 217 to 15


In [64]:
freq = bertmodel.get_topic_info(); freq.head(5)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,3568,-1_eeg_waves_abnormal_slow,"[eeg, waves, abnormal, slow, temporal, consist...",[abnormal eeg focal slowing left fronto tempor...
1,0,4470,0_seizure_sharp_waves_focal,"[seizure, sharp, waves, focal, disorder, tempo...",[abnormal eeg r anterior temporal sharp slow w...
2,1,2358,1_diffuse_dysfunction_cortical_slow,"[diffuse, dysfunction, cortical, slow, encepha...",[diffuse slow waves suggestive diffuse severe ...
3,2,2353,2_normal_eeg_awake_sleep,"[normal, eeg, awake, sleep, drowsiness, waking...","[normal eeg, eeg normal, eeg normal]"
4,3,1686,3_within_limits_normal_eeg,"[within, limits, normal, eeg, limit, age, prob...","[eeg within normal limits, eeg within normal l..."


In [65]:
bertmodel.get_topic_freq().head(10)

,Topic,Count
3,0,4470
0,-1,3568
1,1,2358
2,2,2353
5,3,1686
6,4,218
8,5,55
4,6,46
13,7,30
12,8,27


In [66]:
# Get the most frequent topics
top_topics = bertmodel.get_topic_freq().head(15)

# Loop through the top 10 most frequent topics
for index, row in top_topics.iterrows():
    topic_id = row['Topic']
    frequency = row['Count']
    topic_words = bertmodel.get_topic(topic_id)

    print(f'Topic {topic_id}: (Frequency: {frequency})')
    print(topic_words)
    print()

Topic 0: (Frequency: 4470)
[('seizure', 0.0423712125424733), ('sharp', 0.041591291144571406), ('waves', 0.041024046696353164), ('focal', 0.04064570098572619), ('disorder', 0.04028537059690713), ('temporal', 0.03991178552329808), ('eeg', 0.038582882796947986), ('abnormal', 0.037219198260719816), ('region', 0.0341411032864027), ('epilepsy', 0.027937561344463794)]

Topic -1: (Frequency: 3568)
[('eeg', 0.03796320669647047), ('waves', 0.03676963521236168), ('abnormal', 0.03209509435459221), ('slow', 0.03190167704436723), ('temporal', 0.030416625703100594), ('consistent', 0.028614068902616027), ('background', 0.02843087176887497), ('focal', 0.028235985713648732), ('region', 0.02820018609003759), ('diffuse', 0.027696177742960907)]

Topic 1: (Frequency: 2358)
[('diffuse', 0.09656882808814055), ('dysfunction', 0.07209932891654347), ('cortical', 0.07168566997863073), ('slow', 0.057797815066891446), ('encephalopathy', 0.054280847071301416), ('severe', 0.04873697436897359), ('consistent', 0.047476

In [67]:
bertmodel.visualize_topics()

In [68]:
bertmodel.visualize_barchart()

In [69]:
bertmodel.visualize_heatmap()

In [70]:
bertmodel.visualize_term_rank()

In [71]:
doc_topic = bertmodel.get_document_info(docs)
doc_topic.to_csv("BERT document topic.csv")

In [72]:
bertmodel.save("bert_model")

2024-06-04 10:37:45,401 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.
c:\Users\yijin\anaconda3\envs\thesis\Lib\site-packages\scipy\sparse\_index.py:145: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [73]:
documents = pd.DataFrame({"Document": docs,
                          "ID": range(len(docs)),
                          "Topic": topics})

documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = bertmodel._preprocess_text(documents_per_topic.Document.values)

In [74]:
vectorizer = bertmodel.vectorizer_model
analyzer = vectorizer.build_analyzer()

In [75]:
words = vectorizer.get_feature_names_out
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in bertmodel.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

In [76]:
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')

In [77]:
coherence = coherence_model.get_coherence()

In [78]:
print("Coherence Score:", coherence)

Coherence Score: 0.5890848571943714
